# **HTML Duyarlı Özyineli İşleme ile İngilizceden Almancaya JSON Arayüz Çevirisinin Güncel Dil Modelleriyle Karşılaştırmalı İncelemesi**

Bu çalışma kapsamında, farklı açık kaynaklı dil modelleri ve çeviri yöntemleri kullanılarak JSON tabanlı kullanıcı arayüzü metinlerinin çok dilli çevirisi gerçekleştirildi. Özellikle HTML içeren metinlerin yapısının korunması amacıyla placeholder-temelli ve rekürsif metin işleme teknikleri uygulandı. Denenen modeller arasında Facebook'un NLLB-200 ve M2M100 serisi, Helsinki-NLP’nin Opus-MT modeli gibi dil modelleri yer almaktadır.
Ayrıca bu çalışmada daha önce Türkçeden İngilizceye çeviri yaparken karşılaştırdığım modellerde bazı modeller yüksek maliyetli olmasına rağmen iyi çalışmamıştı o modelleri İngilizceden Almancaya çeviri yaparken de deneme kararı aldım. Ek olarak bu çalışmada İngilizceden Almancaya çeviri modellerini karşılaştırıcam ve optimizasyonlarla güçlendirmeye çalışıcam.

# JSON UI Translation with M2M100 (1.2B) using Placeholder-Based HTML-Aware Text Replacement (EN TO DE).

In [1]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# Model Yükleme
model_name = "facebook/m2m100_1.2B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

src_lang = "en"
tgt_lang = "de"

tokenizer.src_lang = src_lang

# Model yükleme süresi : 50 saniye

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [4]:
# Metin Çeviri Fonksiyonu
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang]
    inputs["forced_bos_token_id"] = forced_bos_token_id
    with torch.no_grad():
        translated_tokens = model.generate(**inputs, max_length=512, forced_bos_token_id=forced_bos_token_id)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [5]:
def translate_text_with_html(text):
    forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)

    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512, forced_bos_token_id=forced_bos_token_id)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [6]:
# JSON'ı Rekürsif Olarak Çevir
# JSON içindeki tüm metin alanlarını iç içe yapılar dahil olmak üzere dolaşarak (recursive) çevirir.
def translate_json(data):
    if isinstance(data, dict):
        result = {}
        for k, v in data.items():
            if isinstance(v, str) and k.lower() == v.lower():
                result[k] = v  # Anahtar ve değer aynıysa çevirme
            else:
                result[k] = translate_json(v)
        return result
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [7]:
input_path = "translated_eng_opus-mt-tr-en.json"
output_path = "translated_de_1.json"

In [8]:
# Dosya oku, Çevir, Kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : 2 dakika 23 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 6.9 GB
# Kullanılan GPU RAM : 5.3 GB

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Çeviri tamamlandı.


# JSON UI Translation with NLLB-200 (3.3B) using Placeholder-Based HTML-Aware Text Replacement (EN TO DE)


Bu Python kodu, Facebook'un `nllb-200-3.3B` modelini kullanarak JSON formatındaki çok katmanlı arayüz metinlerini Türkçeden İngilizceye çevirir. Kod, JSON objelerini rekurzif olarak işleyerek hem sözlük (dict) hem liste (list) yapılarında bulunan tüm metin değerlerini ayıklar ve transformers kütüphanesindeki NLLB modeli ile GPU hızlandırmalı şekilde çeviri yapar. Metin içinde gömülü olan HTML etiketleri, regex ile tespit edilip yer tutucularla maskelenir, böylece modelin bu yapıları bozması engellenir. Çeviri sırasında, NLLB modelinin zorunlu başlangıç tokenı (`forced_bos_token_id`) hedef dil koduna uygun şekilde atanır. Sonuçta, yapısal bütünlük korunarak, sadece içerik metinleri hedef dile yüksek doğrulukla çevrilmiş yeni bir JSON çıktısı oluşturulur. Bu yaklaşım, çok dilli yazılım arayüzü lokalizasyon projelerinde otomatik, yapı korumalı ve yüksek kaliteli çeviri sağlamaya yönelik endüstriyel düzeyde bir çözümdür.

In [1]:
pip install torch transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [17]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# Model Yükleme
model_name = "facebook/nllb-200-3.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

src_lang = "eng_Latn"
tgt_lang = "deu_Latn"

tokenizer.src_lang = src_lang

# Model yükleme süresi : 1 dakika 45 saniye

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [18]:
# Metin Çeviri Fonksiyonu
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang]
    inputs["forced_bos_token_id"] = forced_bos_token_id
    with torch.no_grad():
        translated_tokens = model.generate(**inputs, max_length=512, forced_bos_token_id=forced_bos_token_id)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [23]:
def translate_text_with_html(text):
    forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)

    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512, forced_bos_token_id=forced_bos_token_id)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [20]:
# JSON'ı Rekürsif Olarak Çevir
# JSON içindeki tüm metin alanlarını iç içe yapılar dahil olmak üzere dolaşarak (recursive) çevirir.
def translate_json(data):
    if isinstance(data, dict):
        result = {}
        for k, v in data.items():
            if isinstance(v, str) and k.lower() == v.lower():
                result[k] = v  # Anahtar ve değer aynıysa çevirme
            else:
                result[k] = translate_json(v)
        return result
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [21]:
input_path = "translated_eng_opus-mt-tr-en.json"
output_path = "translated_de_2.json"

In [24]:
# Dosya oku, Çevir, Kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : 1 dakika 47 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 7.9 GB
# Kullanılan GPU RAM : 12.7 GB

Çeviri tamamlandı.


# JSON Translation with HTML Preservation Using Helsinki-NLP Opus-MT (English to German)

Bu kod, verilen JSON dosyasındaki tüm metinleri Türkçe'den İngilizce'ye Helsinki-NLP'nin opus-mt-tr-en modeli ile çevirir. HTML etiketlerini bozmamak için öncelikle metindeki HTML taglerini özel yer tutucularla değiştirir, ardından metni çevirir, son olarak yer tutucuları eski haline getirir. Bu sayede çeviri sırasında HTML yapısı korunur.

In [1]:
pip install transformers sentencepiece torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 135.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

In [2]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# Model ve tokenizer yükle
model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Model Yükleme Süresi : 12 saniye

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [4]:
# HTML etiketlerini bozmadan çeviri için yardımcı fonksiyon
"""
def translate_text_with_html(text):
    # HTML etiketlerini bul ve yer tutucu ile değiştir
    tags = re.findall(r"<[^>]+>", text)
    placeholder_text = text

    for i, tag in enumerate(tags):
        placeholder_text = placeholder_text.replace(tag, f"[TAG{i}]")

    # Çeviri için tokenizasyon ve model çıkışı
    inputs = tokenizer(placeholder_text, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=512)
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Yer tutucuları tekrar orijinal HTML etiketleriyle değiştir
    for i, tag in enumerate(tags):
        translated = translated.replace(f"[TAG{i}]", tag)

    return translated
"""

def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [5]:
# JSON'u rekürsif olarak çeviren fonksiyon
def translate_json(data):
    if isinstance(data, dict):
        return {k: translate_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [translate_json(i) for i in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [6]:
input_path = "translated_eng_opus-mt-tr-en.json"
output_path = "translated_de_3.json"

In [7]:
# JSON dosyasını oku, çevir ve kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : 44 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 2.2 GB
# Kullanılan GPU RAM : 0 GB (CPU kullanıyor model)

Çeviri tamamlandı.


# Sonuç

Bu çalışmada, JSON tabanlı kullanıcı arayüzü metinlerinin çok dilli çevirisi için farklı açık kaynak modelleri denendi. Denenen modeller arasında Facebook'un NLLB-200 (3.3B), M2M100 (1.2B) ve Helsinki-NLP’nin Opus-MT (en-de) yer almaktadır.
Bundan önceki çalışmamda yüksek maliyetli (yüksek parametreli) modellerin çok iyi çalışmadığını fark ettim bu yüzden burada yüksek maliyetli modelleri denemekten kaçındım ve daha önceki çalışmada kullandığım modelin (tr -> en) mevcut çalışmamda İngilizceden Almancaya çeviri yapan bir modelinde olduğunu fark ettim ve onu kullandım yeterince iyi sonuç aldım bundan dolayı yüksek parametreli modelleri çalıştırmaktan vazgeçtim. Ek olarak ilk iki model html etiketlerinde çok iyi (Yardımcı fonksiyon kullanmama rağmen) çalışmadığı gibi çok kötü çeviriler yaptı fakat 3. olan modelim çok iyi sonuçlar verdi.

Karar Verilen Yapının Daha Gelişmiş Github Linki :

Türkçe - İngilizce Çeviri Modelleri Karşılaştırma Raporu Linki :